<a href="https://colab.research.google.com/github/jpedro-rod/ElementosFinitos/blob/main/Elementos_Finitos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Elementos Finitos 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import pandas as pd

Formulação forte de um problema de condução de calor estacionário em uma placa:

$\nabla\cdot\vec{q} + G = 0 \;,\;\; \vec{q} = -k\nabla T$

Formulação fraca:

$∫_{\Omega}[\nabla\cdot\vec{q} + G]\cdot \omega(x,y) \;\; d\Omega = 0, ∀\omega \in U^0$

$∫_{\Omega}[\nabla\cdot\vec{q}\omega + G\omega] \;\; d\Omega = 0, ∀\omega \in U^0$

Pelo Teorema de Green (funcionando como a regra do produto funcionava em 01 dimensão):

$\oint_{\Gamma}\omega\vec{q}\cdot\vec{n} \; d\Gamma - \int_{Ω} \nabla\omega\cdot\vec{q} \;\; dΩ + \int_{\Omega} G\omega \;\; dΩ= 0 \;\;\;\; (I)$

Reescrevendo a 1ª parcela em função das condições de contorno, ter-se-á:

$\oint_{\Gamma}\omega\vec{q}\cdot\vec{n} \; d\Gamma = \int_{\Gamma_q} \omega\vec{q}\cdot\vec{n} \; d\Gamma_q + \int_{\Gamma_T} \omega\vec{q}\cdot\vec{n} \; d\Gamma_T$

Sabe-se ainda que $\omega(x,y)=0$ em $\Gamma_T$, o que simplifica a equação oriunda do Teorema de Green:

$\oint_{\Gamma}\omega\vec{q}\cdot\vec{n} \; d\Gamma = \int_{\Gamma_q} \omega\vec{q}\cdot\vec{n} \; d\Gamma_q \;\;\;\; (II)$

Inserindo $II$ em $I$ e definindo $\overline{q}=\vec{q}\cdot\vec{n}$ finalmente:

$\int_{Ω} \nabla\omega\cdot\vec{q} \;\; dΩ = \int_{\Gamma_q} \omega\overline{q} \;\; dΓ_q + \int_{\Omega} \omega G \;\; dΩ$

$\int_{Ω} \nabla\omega\cdot(-k\nabla T) \;\; dΩ = \int_{\Gamma_q} \omega\overline{q} \;\; dΓ_q + \int_{\Omega} \omega G \;\; dΩ$

Discretizando a fórmula fraca:

$\int_{Ω} -B^{e^T}\omega^{e^T} \cdot k \cdot B^eT^e dΩ^e = \int_{\Gamma_q} N^{e^T}\omega^{e^T}\overline{q} \;\; dΓ_q^e + \int_{\Omega} N^{e^T}\omega^{e^T}G \;\; dΩ^e$

Onde $N^e$ é *matriz* de funções de interpolação (ou de forma) e $B^e$ é matriz das derivadas de $N^e$

$\int_{Ω} -B^{e^T} k B^e \cdot T^e dΩ^e = \int_{\Gamma_q} N^{e^T}\overline{q} \;\; dΓ_q^e + \int_{\Omega} N^{e^T}G \;\; dΩ^e$

## Funções de forma (ou de interpolação)

In [2]:
#1ª ordem:
#N1 = [N11, N12]

def N11(x):
    return 4-x

def N12(x):
    return x-3

def B11(x):
    return -1

def B12(x):
    return 1

#2ª ordem:
#N2 = [N21, N22, N23]

def N21(x):
    return (2/2**2)*(x-2)*(x-3)

def N22(x):
    return (-4/2**2)*(x-1)*(x-3)

def N23(x):
    return (2/2**2)*(x-1)*(x-2)

# B2 = [B21, B22, B23]

def B21(x):
    return (1/2)*(2*x-5)

def B22(x):
    return -2*x+4

def B23(x):
    return (1/2)*(2*x-3)

## Condução de calor em barra delgada

Condução de calor numa barra fina e homogênea com extremidades a temperaturas fixas e submetida a geração de calor de comportamento linear:

$L = 4\:m = 4 \;,\;\; A = 1\:m^2 \;,\;\; K = 5\:W/(°Cm)$

$G(x) = \frac{50}{3}(x+2) \;,\;\; T(x=1) = 100°C \;,\;\; T(x=4) = 0$

$$\int_{Ω} B^{e^T} k B^e \; dx^e \cdot T^e + \int_{\Omega} N^{e^T}G \;\; dx^e = 0$$

In [ ]:
#Parâmetros Físicos

Xo = 1
Xf = 4
A = 1
k = 5
To = 100
Tf = 0

def S(x):
    return (50/3)*(x+2)

#Parâmetros numéricos

ligacoes = 4
malha = np.linspace(Xo,Xf,ligacoes)
nElementos = len(malha) - 1
ordemElemento = 2   #1 ou 2

'''
Se = np.array([[S]])

#Vetores de forma

Net = np.array([[N21,N22,N23]])
Ne = np.transpose(Net)

Bet = np.array([[B21,B22,B23]])
Be = np.transpose(Bet)

#Matrizes elementares

dK = Be@Bet   #Certo seria Be @ K @ Bet, mas como em uma dimensão K é escalar, o produto se torna comutativo
dG = Se@Net
'''

#Montando matrizes elementares e integrando

##Função de integração a ser usada

def INT(f,Xo,Xf):
    return sp.integrate.quad(f,Xo,Xf)[0]

##Produtos das funções de forma 2ª ordem

if ordemElemento == 2:
    K11 = lambda x : B21(x)*k*B21(x)
    K12 = lambda x : B21(x)*k*B22(x)
    K13 = lambda x : B21(x)*k*B23(x)
    K21 = lambda x : B22(x)*k*B21(x)
    K22 = lambda x : B22(x)*k*B22(x)
    K23 = lambda x : B22(x)*k*B23(x)
    K31 = lambda x : B23(x)*k*B21(x)
    K32 = lambda x : B23(x)*k*B22(x)
    K33 = lambda x : B23(x)*k*B23(x)

    G1 = lambda x : S(x)*N21(x)
    G2 = lambda x : S(x)*N22(x)
    G3 = lambda x : S(x)*N23(x)

##Integrando e montando

    listaKe = np.empty((3,3,nElementos))
    listaGe = np.empty((1,3,nElementos))

    for e in range(nElementos):
        
        Ke = np.array([[INT(K11,malha[e],malha[e+1]),INT(K12,malha[e],malha[e+1]),INT(K13,malha[e],malha[e+1])],
                    [INT(K21,malha[e],malha[e+1]),INT(K22,malha[e],malha[e+1]),INT(K23,malha[e],malha[e+1])],
                    [INT(K31,malha[e],malha[e+1]),INT(K32,malha[e],malha[e+1]),INT(K33,malha[e],malha[e+1])]])         

        Ge = np.array([[INT(G1,malha[e],malha[e+1]),INT(G2,malha[e],malha[e+1]),INT(G3,malha[e],malha[e+1])]])
        
        listaKe[:,:,e] = Ke
        listaGe[:,:,e] = Ge

##Produtos das funções de forma 1ª ordem

elif ordemElemento == 1:
    K11 = lambda x : B11(x)*k*B11(x)
    K12 = lambda x : B11(x)*k*B12(x)
    K21 = lambda x : B12(x)*k*B11(x)
    K22 = lambda x : B12(x)*k*B12(x)

    G1 = lambda x : S(x)*N11(x)
    G2 = lambda x : S(x)*N12(x)

##Integrando e montando

    listaKe = np.empty((2,2,nElementos))
    listaGe = np.empty((1,2,nElementos))

    for e in range(0,nElementos,2):
        Ke = np.array([[INT(K11,malha[e],malha[e+1]),INT(K12,malha[e],malha[e+1])],
                    [INT(K21,malha[e],malha[e+1]),INT(K22,malha[e],malha[e+1])]])         

        Ge = np.array([[INT(G1,malha[e],malha[e+1]),INT(G2,malha[e],malha[e+1]),INT(G3,malha[e],malha[e+1])]])
        
        listaKe[:,:,e] = Ke
        listaGe[:,:,e] = Ge

#Matriz de conectividade

M = np.empty((nElementos,3))

for elemento in range(nElementos):
    for noLocal in range(3):

        M[elemento,noLocal] = elemento*2 + noLocal

print(M)

#Matrizes globais

K = np.zeros((nElementos*3 - nElementos,nElementos*3 - nElementos))
G = np.zeros(nElementos*3 - nElementos)

print(len(listaKe[0,0,:]))
for indice in range(len(listaKe[0,0,:])):
    
    Kn = np.empty((nElementos*3 - nElementos,nElementos*3 - nElementos))
    
    Kn[indice*3:indice*3+3,indice*3:indice*3+3] = listaKe[:,:,indice]

    K = K + Kn

print(K)

In [101]:
A = np.array([[1,1,1],[1,1,1]])
B = np.array([[1,1],[2,2],[3,3]])
C = np.array([[1],[2],[3]])
Ct = np.transpose(C)

D = np.array([[1,2,3]])
Dt = np.transpose(D)
print(D.shape)
print(Dt.shape)
print(D)
print(Dt)
print(Dt @ D)


(1, 3)
(3, 1)
[[1 2 3]]
[[1]
 [2]
 [3]]
[[1 2 3]
 [2 4 6]
 [3 6 9]]
